In [0]:
import os
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from keras.initializers import glorot_uniform
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model, load_model
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#load in the training data and labels
X_train = np.load("/content/drive/My Drive/competition_data/X_train.npy")
Y_train = np.load("/content/drive/My Drive/competition_data/Y_train.npy")

#load in the validation data and labels
X_val = np.load("/content/drive/My Drive/competition_data/X_val.npy")
Y_val = np.load("/content/drive/My Drive/competition_data/Y_val.npy")

In [4]:
#default settings used
batch_size = 128
epochs = 10
image_size = X_train.shape[1:]
output_size = 12
print(image_size)

(161, 99, 1)


In [0]:
#scale the X data
X_train_scaled = X_train * (1 / 255)
X_val_scaled = X_val * (1  / 255)

In [0]:
#one-hot encode the Y data
Y_train = keras.utils.to_categorical(Y_train,output_size)
Y_val = keras.utils.to_categorical(Y_val,output_size)

In [0]:
#building the identity block of the ResNet

def identity_block(X,f,filters,stage,block):
    
    #this is the function for the identity block
    #X is the input tensor
    #f is the filter shape for the middle block
    #filters is the list containing the filter sizes (int)
    #stage names the layer relative to the position in the network
    #block is used to name the layers 
    
    #name definition
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    
    #filters 1,2,3
    f1, f2, f3 = filters
    
    #initial value
    X_shortcut = X
    
    #define the first component block 
    X = Conv2D(filters=f1, kernel_size=(1,1),strides=(1,1),padding="valid",name=conv_name_base+"2a",kernel_initializer= glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base + "2a")(X)
    X = Activation("relu")(X)
    
    #second component block 
    X = Conv2D(filters=f2, kernel_size=(f,f),strides=(1,1),padding="same",name=conv_name_base+"2b",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base + "2b")(X)
    X = Activation("relu")(X)
    
    #third component block 
    X = Conv2D(filters=f3,kernel_size=(1,1),strides=(1,1),padding="valid",name=conv_name_base+"2c",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base + "2c")(X)
    
    #make the connection at the add block 
    X = Add()([X,X_shortcut])
    #finish with the relu activation layer
    X = Activation("relu")(X)
    
    #return the new value of X
    return X

In [0]:
def convolution_block(X,f,filters,stage,block,s=2):
    
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    
    f1, f2, f3 = filters
    
    #initialize the value of X
    X_shortcut = X
    
    #the branch from the main path
    X_filt3 = Conv2D(f3,(1,1),strides=(s,s),name=conv_name_base+"1",kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_filt3 = BatchNormalization(axis=3,name=bn_name_base + "1")(X_filt3)
    
    #main path
    
    #block 1
    X = Conv2D(f1,kernel_size=(1,1),strides=(s,s),name=conv_name_base+"2a",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+"2a")(X)
    X = Activation("relu")(X)
    
    #block 2
    X = Conv2D(f2,kernel_size=(f,f),strides=(1,1),padding="same",name=conv_name_base+"2b",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+"2b")(X)
    X = Activation("relu")(X)
    
    #block 3
    X = Conv2D(f3,kernel_size=(1,1),strides=(1,1),padding="valid",name=conv_name_base+"2c",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+"2c")(X)
    
    #add the branch and the main path together
    X = Add()([X,X_filt3])
    
    #finish with a relu
    X = Activation("relu")(X) 
    
    #return the new value of X
    return X

In [0]:
def ResNet50(input_shape=(161,99,1), num_classes=6):
    
    X_input = Input(input_shape)
    
    #padding
    X = ZeroPadding2D((3,3))(X_input)
    
    #stage1
    X = Conv2D(128, (7,7),strides=(2,2),name="conv1",kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name="bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3,3),strides=(2,2))(X)

    X = Dropout(0.25)(X)
    
    #stage2
    X = convolution_block(X,f=3,filters=[64,64,256],stage=2,s=1,block="a")
    X = identity_block(X,f=3,filters=[64,64,256],stage=2,block="b")
    X = identity_block(X,f=3,filters=[64,64,256],stage=2,block="c")

    X = Dropout(0.25)(X)
    
    #stage3
    X = convolution_block(X,f=3,filters=[128,128,512],stage=3,s=2,block="a")
    X = identity_block(X,f=3,filters=[128,128,512],stage=3,block="b")
    X = identity_block(X,f=3,filters=[128,128,512],stage=3,block="c")
    X = identity_block(X,f=3,filters=[128,128,512],stage=3,block="d")

    X = Dropout(0.25)(X)

    #stage4
    X = convolution_block(X,f=3,filters=[256,256,1024],stage=4,s=2,block="a")
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block="b")
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block="c")
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block="d")
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block="e")
    X = identity_block(X,f=3,filters=[256,256,1024],stage=4,block="f")

    X = Dropout(0.25)(X)
    
    #stage5
    X = convolution_block(X,f=3,filters=[512,512,2048],stage=5,s=2,block="a")
    X = identity_block(X,f=3,filters=[512,512,2048],stage=5,block="b")
    X = identity_block(X,f=3,filters=[512,512,2048],stage=5,block="c")
    
    X = Dropout(0.25)(X)

    #stage6
    X = AveragePooling2D(pool_size=(2,2),name="avg_pool")(X)
    X = Flatten()(X)
    X = Dense(num_classes,activation="softmax",name="fc"+str(num_classes))(X)
    
    model = Model(inputs=X_input,outputs=X,name="ResNet50")
    
    return model

In [35]:
model = ResNet50(input_shape=(161,99,1),num_classes=12)

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["acc"])

model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 161, 99, 1)   0                                            
__________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D (None, 167, 105, 1)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 81, 50, 128)  6400        zero_padding2d_7[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 81, 50, 128)  512         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
Y_integers = np.argmax(Y_train, axis=1)
class_weights = compute_class_weight('balanced', np.unique(Y_integers), Y_integers)
d_class_weights = dict(enumerate(class_weights))

In [37]:
history = model.fit(X_train_scaled,Y_train,epochs=20,validation_data=(X_val_scaled,Y_val),
                    batch_size=128,shuffle=True,class_weight=d_class_weights)

Train on 59531 samples, validate on 7200 samples
Epoch 1/20
59531/59531 [==============================] - 169s 3ms/step - loss: 1.8271 - acc: 0.2759 - val_loss: 1.1592 - val_acc: 0.5794
Epoch 2/20
59531/59531 [==============================] - 147s 2ms/step - loss: 0.5291 - acc: 0.6946 - val_loss: 1.3170 - val_acc: 0.6453
Epoch 3/20
59531/59531 [==============================] - 147s 2ms/step - loss: 0.3871 - acc: 0.7754 - val_loss: 0.4419 - val_acc: 0.8542
Epoch 4/20
59531/59531 [==============================] - 147s 2ms/step - loss: 0.2662 - acc: 0.8438 - val_loss: 0.5062 - val_acc: 0.8268
Epoch 5/20
59531/59531 [==============================] - 146s 2ms/step - loss: 0.3053 - acc: 0.8207 - val_loss: 0.3801 - val_acc: 0.8668
Epoch 6/20
59531/59531 [==============================] - 146s 2ms/step - loss: 0.2058 - acc: 0.8784 - val_loss: 0.8407 - val_acc: 0.7318
Epoch 7/20
59531/59531 [==============================] - 146s 2ms/step - loss: 0.1763 - acc: 0.8959 - val_loss: 0.4449 - v